# Clean Austin-3-1-1 Data 

<img src="imgs/cleaning.jpg" width = "500" align="center"/>

In [1]:
import numpy as np # Linear algebra lib
import pandas as pd # Data analysis lib

# Removes rows and columns truncation of '...'
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Load data

In [2]:
# Toggle Comments to run
!mkdir -p 'raw_data'
!rm -f raw_data/'311_Unified_Data.csv'
!wget 'https://austin-311-data.s3.us-east-2.amazonaws.com/311_Unified_Data.csv' -P raw_data
!ls -lh raw_data
!head raw_data/'311_Unified_Data.csv'
!tail raw_data/'311_Unified_Data.csv'
!wc -l raw_data/'311_Unified_Data.csv'

--2019-07-31 21:31:38--  https://austin-311-data.s3.us-east-2.amazonaws.com/311_Unified_Data.csv
Resolving austin-311-data.s3.us-east-2.amazonaws.com (austin-311-data.s3.us-east-2.amazonaws.com)... 52.219.80.232
Connecting to austin-311-data.s3.us-east-2.amazonaws.com (austin-311-data.s3.us-east-2.amazonaws.com)|52.219.80.232|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 257255577 (245M) [text/csv]
Saving to: ‘raw_data/311_Unified_Data.csv’

311_Unified_Data.cs 100%[===================>] 245.34M  1.89MB/s    in 3m 20s  

2019-07-31 21:34:59 (1.23 MB/s) - ‘raw_data/311_Unified_Data.csv’ saved [257255577/257255577]

total 247M
-rw-r--r-- 1 turtle turtle 246M Jul 30 20:56 311_Unified_Data.csv
drwxr-xr-x 2 turtle turtle 4.0K Jul 30 20:46 ACS_Surveys_Austin
-rw-r--r-- 1 turtle turtle 785K Jul 30 20:56 ACS_Surveys_Austin.zip
Service Request (SR) Number,SR Type Code,SR Description,Owning Department,Method Received,SR Status,Status Change Date,Created Date,Last Upda

## Clean Data

Start by loading the dataset

In [3]:
df = pd.read_csv('raw_data/311_Unified_Data.csv', low_memory=False)
df.head()

,Service Request (SR) Number,SR Type Code,SR Description,Owning Department,Method Received,SR Status,Status Change Date,Created Date,Last Update Date,Close Date,SR Location,Street Number,Street Name,City,Zip Code,County,State Plane X Coordinate,State Plane Y Coordinate,Latitude Coordinate,Longitude Coordinate,(Latitude.Longitude),Council District,Map Page,Map Tile
0,16-00107769,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,Duplicate (closed),05/06/2016 12:11:57 PM,05/06/2016 12:01:46 PM,05/06/2016 12:11:57 PM,05/06/2016 12:11:57 PM,"WINDSOR RD & EXPOSITION BLVD, AUSTIN, TX 78703",NaN,WINDSOR RD & EXPOSITION BLVD,AUSTIN,78703.0,TRAVIS,3106038.49745799,1.008098e+07,30.296699,-97.768264,"(30.29669887, -97.76826374)",10.0,584C,MH24
1,16-00108244,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,Duplicate (closed),05/06/2016 07:51:27 PM,05/06/2016 07:51:27 PM,05/06/2016 07:51:27 PM,05/06/2016 07:51:27 PM,"6001 MANCHACA RD, AUSTIN, TX 78745",6001,MANCHACA,AUSTIN,78745.0,TRAVIS,3096240.5,1.005019e+07,30.212695,-97.801521,"(30.2126949, -97.8015215)",5.0,643M,MF17
2,16-00108269,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,Duplicate (closed),05/06/2016 08:22:56 PM,05/06/2016 08:22:56 PM,05/06/2016 08:22:56 PM,05/06/2016 08:22:56 PM,"6001 MANCHACA RD, AUSTIN, TX 78745",6001,MANCHACA,AUSTIN,78745.0,TRAVIS,3096240.5,1.005019e+07,30.212695,-97.801521,"(30.2126949, -97.8015215)",5.0,643M,MF17
3,16-00324071,SWSDEADA,ARR Dead Animal Collection,Austin Resource Recovery,Phone,Closed,12/15/2016 09:05:43 AM,12/15/2016 06:41:40 AM,12/15/2016 09:05:43 AM,12/15/2016 09:05:43 AM,"2200 E OLTORF ST, AUSTIN, TX 78741",2200,OLTORF,AUSTIN,78741.0,TRAVIS,3118116.50749136,1.005705e+07,30.230164,-97.731776,"(30.23016411, -97.73177647)",3.0,615X,MJ19
4,16-00108062,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,Duplicate (closed),05/10/2016 04:56:52 PM,05/06/2016 05:03:45 PM,05/10/2016 04:56:52 PM,05/10/2016 04:56:52 PM,"8401 N CAPITAL OF TEXAS HWY NB, AUSTIN, TX 78759",8401,CAPITAL OF TEXAS,AUSTIN,78759.0,TRAVIS,3105863.35797435,1.011309e+07,30.384989,-97.766471,"(30.38498865, -97.76647071)",10.0,494L,MH32


In [4]:
print('This dataset has number of rows {}, number of cols {}'.format(df.shape[0], df.shape[1]))

This dataset has number of rows 747986, number of cols 24


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747986 entries, 0 to 747985
Data columns (total 24 columns):
Service Request (SR) Number    743752 non-null object
SR Type Code                   743751 non-null object
SR Description                 743751 non-null object
Owning Department              743751 non-null object
Method Received                743751 non-null object
SR Status                      743751 non-null object
Status Change Date             737819 non-null object
Created Date                   737819 non-null object
Last Update Date               737819 non-null object
Close Date                     695220 non-null object
SR Location                    738374 non-null object
Street Number                  592926 non-null object
Street Name                    738287 non-null object
City                           737345 non-null object
Zip Code                       735561 non-null float64
County                         731471 non-null object
State Plane X Coordinate

For our analysis we don't need following columns:
 
  - `Service Request (SR) Number`
  - `Status Change Date`
  - `SR Status`
  - `Last Update Date`
  - `Close Date`
  - `Map Page`
  - `Map Tile`
  - `State Plane X Coordinate`
  - `State Plane Y Coordinate`
  - `Street Number`
  - `Street Name`
  - `SR Location`
  - `(Latitude.Longitude)`
  - `Council District`

### Drop unecessary columns and empty rows

In [6]:
columns = ['Council District', 'Map Page', 'Map Tile', 'Service Request (SR) Number', 'Status Change Date', 'Last Update Date', 'Close Date', 'SR Status', 'SR Location', 'Street Number', 'Street Name', 'State Plane X Coordinate', 'State Plane Y Coordinate', '(Latitude.Longitude)']
df = df.drop(columns, axis=1)
df.head()

,SR Type Code,SR Description,Owning Department,Method Received,Created Date,City,Zip Code,County,Latitude Coordinate,Longitude Coordinate
0,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 12:01:46 PM,AUSTIN,78703.0,TRAVIS,30.296699,-97.768264
1,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 07:51:27 PM,AUSTIN,78745.0,TRAVIS,30.212695,-97.801521
2,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 08:22:56 PM,AUSTIN,78745.0,TRAVIS,30.212695,-97.801521
3,SWSDEADA,ARR Dead Animal Collection,Austin Resource Recovery,Phone,12/15/2016 06:41:40 AM,AUSTIN,78741.0,TRAVIS,30.230164,-97.731776
4,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 05:03:45 PM,AUSTIN,78759.0,TRAVIS,30.384989,-97.766471


### Check for missing values and drop rows that are missing important info.

In [7]:
df.isnull().sum()

SR Type Code             4235
SR Description           4235
Owning Department        4235
Method Received          4235
Created Date            10167
City                    10641
Zip Code                12425
County                  16515
Latitude Coordinate      9686
Longitude Coordinate     9686
dtype: int64

So, there are lot missing values, mostly due to empty rows, let's drop those rows now.

In [8]:
df = df.dropna(how='all') # how='all' drops rows that have all NaN values, whereas, 'any' will drop any row that has NaN present
df.isnull().sum()

SR Type Code                0
SR Description              0
Owning Department           0
Method Received             0
Created Date             5932
City                     6406
Zip Code                 8190
County                  12280
Latitude Coordinate      5451
Longitude Coordinate     5451
dtype: int64

After dropping rows that had all `NaN` values, now we can drop those that having missing valuable information needed to stratify complaints by their location info. 



#### Missing all location data (i.e. Zip Code and County and City)

Let's drop rows that contain all of the missing location info.

In [9]:
print('Before dimensions: ', df.shape)
df = df.loc[df[['City', 'Zip Code', 'County']].notnull().values.any(axis=1)]
print('After dimensions: ', df.shape)
df.head()

Before dimensions:  (743751, 10)
After dimensions:  (738061, 10)


,SR Type Code,SR Description,Owning Department,Method Received,Created Date,City,Zip Code,County,Latitude Coordinate,Longitude Coordinate
0,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 12:01:46 PM,AUSTIN,78703.0,TRAVIS,30.296699,-97.768264
1,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 07:51:27 PM,AUSTIN,78745.0,TRAVIS,30.212695,-97.801521
2,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 08:22:56 PM,AUSTIN,78745.0,TRAVIS,30.212695,-97.801521
3,SWSDEADA,ARR Dead Animal Collection,Austin Resource Recovery,Phone,12/15/2016 06:41:40 AM,AUSTIN,78741.0,TRAVIS,30.230164,-97.731776
4,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 05:03:45 PM,AUSTIN,78759.0,TRAVIS,30.384989,-97.766471


In [10]:
df.isnull().sum()

SR Type Code               0
SR Description             0
Owning Department          0
Method Received            0
Created Date            5889
City                     716
Zip Code                2500
County                  6590
Latitude Coordinate       70
Longitude Coordinate      70
dtype: int64

After eliminating rows with no location information, we can now dive into missing `Latitude Longitude` coordinate values, as they are important to our analysis.

#### Drop Missing Lat., Long. Coordinates

In [12]:
print('Before dimensions: ', df.shape)
df = df.loc[df[['Latitude Coordinate', 'Longitude Coordinate']].notnull().values.any(axis=1)]
print('After dimensions: ', df.shape)
df.head()

Before dimensions:  (738061, 10)
After dimensions:  (737991, 10)


,SR Type Code,SR Description,Owning Department,Method Received,Created Date,City,Zip Code,County,Latitude Coordinate,Longitude Coordinate
0,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 12:01:46 PM,AUSTIN,78703.0,TRAVIS,30.296699,-97.768264
1,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 07:51:27 PM,AUSTIN,78745.0,TRAVIS,30.212695,-97.801521
2,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 08:22:56 PM,AUSTIN,78745.0,TRAVIS,30.212695,-97.801521
3,SWSDEADA,ARR Dead Animal Collection,Austin Resource Recovery,Phone,12/15/2016 06:41:40 AM,AUSTIN,78741.0,TRAVIS,30.230164,-97.731776
4,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 05:03:45 PM,AUSTIN,78759.0,TRAVIS,30.384989,-97.766471


#### Drop Missing Zipcodes

> NOTE: We can actually reverse geocode location info like, Street Address, City, County, Zip Code, and State using something like [Google Maps Reverse Geocode API](https://developers.google.com/maps/documentation/geocoding/start#reverse), and have that fill in the missing information. 


In [13]:
print('Before dimensions: ', df.shape)
df = df.loc[df['Zip Code'].notnull().values]
print('After dimensions: ', df.shape)
df.head()

Before dimensions:  (737991, 10)
After dimensions:  (735502, 10)


,SR Type Code,SR Description,Owning Department,Method Received,Created Date,City,Zip Code,County,Latitude Coordinate,Longitude Coordinate
0,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 12:01:46 PM,AUSTIN,78703.0,TRAVIS,30.296699,-97.768264
1,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 07:51:27 PM,AUSTIN,78745.0,TRAVIS,30.212695,-97.801521
2,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 08:22:56 PM,AUSTIN,78745.0,TRAVIS,30.212695,-97.801521
3,SWSDEADA,ARR Dead Animal Collection,Austin Resource Recovery,Phone,12/15/2016 06:41:40 AM,AUSTIN,78741.0,TRAVIS,30.230164,-97.731776
4,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 05:03:45 PM,AUSTIN,78759.0,TRAVIS,30.384989,-97.766471


### Filter out rows that match Zipcodes from Austin and Travis County

Let's use official City of Austin's to figure out Zip Codes under city territory.

![](https://i.imgur.com/kdG5bxm.png)


All of the polygons with zipcodes intersecting under blue-shaded region are under `City of Austin's` jurisdiction, but it's hazy at best, so we will keep all those complaints, under those zipcodes. 

In [14]:
austin_zipcodes = """
78701,78702,78703,78704,78705,
78721,78722,78723,78724,78725,
78726,78727,78728,78729,78730,
78731,78732,78733,78734,78735,
78736,78737,78738,78739,78741,
78742,78744,78745,78746,78747,
78748,78749,78750,78751,78752,
78753,78754,78756,78757,78758,
78759,78610,78617,78653,78660
""".split(',')
austin_zipcodes = [x.strip() for x in austin_zipcodes]
print('Before dimensions: ', df.shape)
df = df.loc[df['Zip Code'].isin(np.array(austin_zipcodes))]
print('After dimensions: ', df.shape)
df.head()

Before dimensions:  (735502, 10)
After dimensions:  (721244, 10)


,SR Type Code,SR Description,Owning Department,Method Received,Created Date,City,Zip Code,County,Latitude Coordinate,Longitude Coordinate
0,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 12:01:46 PM,AUSTIN,78703.0,TRAVIS,30.296699,-97.768264
1,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 07:51:27 PM,AUSTIN,78745.0,TRAVIS,30.212695,-97.801521
2,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 08:22:56 PM,AUSTIN,78745.0,TRAVIS,30.212695,-97.801521
3,SWSDEADA,ARR Dead Animal Collection,Austin Resource Recovery,Phone,12/15/2016 06:41:40 AM,AUSTIN,78741.0,TRAVIS,30.230164,-97.731776
4,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 05:03:45 PM,AUSTIN,78759.0,TRAVIS,30.384989,-97.766471


In [15]:
df['County'].value_counts(dropna=False)

TRAVIS        702371
WILLIAMSON     13758
NaN             5020
HAYS              76
BASTROP           19
Name: County, dtype: int64

### Clean `city` column

#### Strip leading and trailing whitespaces first from entire dataframe

##### Before Stripping whitespaces

In [16]:
df.isnull().sum()

SR Type Code               0
SR Description             0
Owning Department          0
Method Received            0
Created Date            5732
City                     326
Zip Code                   0
County                  5020
Latitude Coordinate        0
Longitude Coordinate       0
dtype: int64

In [17]:
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df.replace('', np.NaN, inplace=True)

##### After stripping whitespaces

In [18]:
df.isnull().sum()

SR Type Code               0
SR Description             0
Owning Department          0
Method Received            0
Created Date            5732
City                     326
Zip Code                   0
County                  5020
Latitude Coordinate        0
Longitude Coordinate       0
dtype: int64

#### Let's start by fixing casing of `City` and `County` columns to `Title Case`, and fix any misspellings.

In [19]:
df['County'] = df['County'].str.title()
df['County'].value_counts(dropna=False).sort_values(ascending=False)

Travis        702371
Williamson     13758
NaN             5020
Hays              76
Bastrop           19
Name: County, dtype: int64

In [20]:
df['City'] = df['City'].str.title()
df['City'].value_counts(dropna=False).sort_values(ascending=False)

Austin                      710153
Pflugerville                  3013
Del Valle                     2232
Manor                         1723
Austin 5 Etj                  1111
Bee Cave                       442
Sunset Valley                  421
NaN                            326
Lakeway                        315
West Lake Hills                261
Mustang Ridge                  170
West Lake Hill                 167
Buda                           142
Rollingwood                    136
Creedmoor                      116
Travis                         111
Webberville                     88
Other                           46
The Hills                       43
Round Rock                      42
Cedar Park                      23
Austn                           23
San Leanna                      18
Dripping Springs                15
Dripping Sprin                  15
Hays                            11
Manchaca                         9
Lago Vista                       9
West Lake Hil       

Not so surprisingly we have so many typos, and misspellings, which we can fix, but surprisingly there are calls from cities that are miles apart like `Houston`, `Dallas`, etc.., so let's keep those that are only from `City of Austin` in Travis County which is our main focus.

##### Austin and it's Extraterritorial Jurisdiction

Another thing to note is `Austin's 5 ETJ's` (Extraterritorial Jurisdiction) refers to cities, is the legal capability of a municipality to exercise authority beyond the boundaries of its incorporated area. In the US, Texas is one of the states that by law allow cities to claim ETJ to contiguous land beyond their city limits.  Austin’s ETJ currently extends into 4 counties including Williamson, Travis, Hays, and Bastrop.

In [21]:
df.loc[df['City'].str.contains('Austin 5 Etj', na=False), 'County'].value_counts(dropna=False)

Travis     1089
Hays          9
NaN           7
Bastrop       6
Name: County, dtype: int64

In [22]:
df.loc[df['City'].str.contains('Austin 5 Etj', na=False), 'Zip Code'].value_counts(dropna=False).sort_index()

78610.0     26
78617.0    171
78653.0    315
78660.0      2
78724.0     16
78725.0    328
78733.0     10
78734.0    103
78735.0     31
78736.0     25
78737.0     43
78738.0      5
78747.0     35
78758.0      1
Name: Zip Code, dtype: int64

#### Fix `Austin` related typos and misspellings

In [23]:
df.loc[(df['City'] != 'Austin 5 Etj') & df['City'].str.startswith('Aus', na=False).values, 'City'] = 'Austin'
df['City'].value_counts(dropna=False).sort_values(ascending=False)

Austin                      710194
Pflugerville                  3013
Del Valle                     2232
Manor                         1723
Austin 5 Etj                  1111
Bee Cave                       442
Sunset Valley                  421
NaN                            326
Lakeway                        315
West Lake Hills                261
Mustang Ridge                  170
West Lake Hill                 167
Buda                           142
Rollingwood                    136
Creedmoor                      116
Travis                         111
Webberville                     88
Other                           46
The Hills                       43
Round Rock                      42
Cedar Park                      23
San Leanna                      18
Dripping Sprin                  15
Dripping Springs                15
Hays                            11
Lago Vista                       9
Manchaca                         9
West Lake Hil                    8
Leander             

Looks like there is considerable complaints from surrounding territories that are under City of Austin's jurisdiction.

In [24]:
df['County'].value_counts(dropna=False).sort_index()

Bastrop           19
Hays              76
Travis        702371
Williamson     13758
NaN             5020
Name: County, dtype: int64

In [25]:
df[['County', 'City']].groupby(['County', 'City']).size()

County      City                    
Bastrop     Austin                           1
            Austin 5 Etj                     6
            Bastrop                          1
            Del Valle                        4
            Other                            3
Hays        Austin                           7
            Austin 5 Etj                     9
            Bear Creek                       2
            Buda                            25
            Dripping Sprin                   7
            Dripping Springs                13
            Hays                             7
            Mountain City                    1
Travis      Atx                              2
            Au                               1
            Austin                      691627
            Austin 5 Etj                  1089
            Bee Cave                       440
            Buda                           101
            Cedar Creek                      1
            Cedar Park 

Looks like there are discrepancies in the data as `Austin` is in `Williamson`, `Bastrop`, and `Hays`, while the zipcodes we filtered only pertained to `City of Austin` territories in `Travis` county.

As we noted earlier, using Google Maps API to reverse geocode to actual data would need to be done to assert the validity of `Zip Code`, `City` and `County`. 

#### Drop duplicate rows

Let's check first to see if we have duplicate rows or not,

In [26]:
df[df.duplicated()].sample(10)

,SR Type Code,SR Description,Owning Department,Method Received,Created Date,City,Zip Code,County,Latitude Coordinate,Longitude Coordinate
428381,CODECOMP,Austin Code - Request Code Officer,Austin Code Department,Phone,NaN,Austin,78758.0,Travis,30.384168,-97.707233
429425,LOUDMUS1,Loud Commercial Music,Economic Development Department,Spot311 Interface,NaN,Austin,78701.0,Travis,30.269579,-97.749178
345783,HHSGRAFF,Graffiti Abatement,Health & Human Services,Mass Entry,11/30/2015 12:00:00 AM,Austin,78702.0,Travis,30.255428,-97.707451
120188,HHSGRAFF,Graffiti Abatement,Health & Human Services,Mass Entry,04/01/2014 12:00:00 AM,Austin,78704.0,Travis,30.250651,-97.754705
360240,HHSGRAFF,Graffiti Abatement,Health & Human Services,Mass Entry,01/25/2016 12:00:00 AM,Austin,78704.0,Travis,30.228352,-97.761687
161133,HHSGRAFF,Graffiti Abatement,Health & Human Services,Mass Entry,08/19/2014 12:00:00 AM,Austin,78701.0,Travis,30.263823,-97.745874
426787,LOUDMUS1,Loud Commercial Music,Economic Development Department,Phone,NaN,Austin,78701.0,Travis,30.267085,-97.740104
674242,CODECOMP,Austin Code - Request Code Officer,Austin Code Department,Phone,10/01/2018 04:41:00 PM,Austin,78701.0,Travis,30.264185,-97.743515
671459,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,09/25/2018 05:12:00 PM,Austin,78741.0,Travis,30.233678,-97.723496
174351,HHSGRAFF,Graffiti Abatement,Health & Human Services,Mass Entry,09/17/2014 12:00:00 AM,Austin,78757.0,Travis,30.339646,-97.730880


Indeed, there are duplicate rows, let's drop them.

In [27]:
print('Before dimensions: ', df.shape)
df = df.drop_duplicates()
print('After dimensions: ', df.shape)
df[df.duplicated()].head() # Check again to see if duplicate rows are dropped

Before dimensions:  (721244, 10)
After dimensions:  (719528, 10)


,SR Type Code,SR Description,Owning Department,Method Received,Created Date,City,Zip Code,County,Latitude Coordinate,Longitude Coordinate


Let's now drop the `City` and `County` columns, since we are going to use only `Zip Code` and `Latitude Longitude` columns to map our plots.

#### Drop `City`, and `County` columns

In [28]:
df = df.drop(['City', 'County'], axis=1)
df = df.reset_index(drop=True)
df.head()

,SR Type Code,SR Description,Owning Department,Method Received,Created Date,Zip Code,Latitude Coordinate,Longitude Coordinate
0,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 12:01:46 PM,78703.0,30.296699,-97.768264
1,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 07:51:27 PM,78745.0,30.212695,-97.801521
2,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 08:22:56 PM,78745.0,30.212695,-97.801521
3,SWSDEADA,ARR Dead Animal Collection,Austin Resource Recovery,Phone,12/15/2016 06:41:40 AM,78741.0,30.230164,-97.731776
4,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,05/06/2016 05:03:45 PM,78759.0,30.384989,-97.766471


#### Drop rows that have no `Created Date`

For our analysis to figure when the `Complaint` was noted, we need `Created Date` to be non-null containing column, so let's drop those rows.

In [29]:
df[df['Created Date'].isnull().values].head()

,SR Type Code,SR Description,Owning Department,Method Received,Created Date,Zip Code,Latitude Coordinate,Longitude Coordinate
404440,DRCHANEL,Channels/Creeks/Drainage Easement,Watershed Protection,Phone,NaN,78748.0,30.145042,-97.807026
404441,LEAD1,Neighborhood Home Programs,Neighborhood Housing & Community Development,Phone,NaN,78728.0,30.439929,-97.671150
404442,LEAD1,Neighborhood Home Programs,Neighborhood Housing & Community Development,Phone,NaN,78741.0,30.230990,-97.705644
404664,SBSIDERE,Sidewalk Repair,Public Works,Spot311 Interface,NaN,78701.0,30.266369,-97.740305
404671,HHSGRAFF,Public Health - Graffiti Abatement,Public Health,Phone,NaN,78752.0,30.321899,-97.695546


In [30]:
df = df.loc[df['Created Date'].notnull().values]
df.isnull().sum()

SR Type Code            0
SR Description          0
Owning Department       0
Method Received         0
Created Date            0
Zip Code                0
Latitude Coordinate     0
Longitude Coordinate    0
dtype: int64

##### Reset the index

In [31]:
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714494 entries, 0 to 714493
Data columns (total 8 columns):
SR Type Code            714494 non-null object
SR Description          714494 non-null object
Owning Department       714494 non-null object
Method Received         714494 non-null object
Created Date            714494 non-null object
Zip Code                714494 non-null float64
Latitude Coordinate     714494 non-null float64
Longitude Coordinate    714494 non-null float64
dtypes: float64(3), object(5)
memory usage: 43.6+ MB


### Some Feature Engineering

In [32]:
# Change Zip Code from float to integer data type
df['Zip Code'] = df['Zip Code'].astype(int)

# Change Created Date to datetime object, so we can extract date, month, year, and hour
df['Created Date'] = pd.to_datetime(df['Created Date'], format='%m/%d/%Y %I:%M:%S %p')

# Add month, year columns
df['Incident Year'] = df['Created Date'].dt.year
df['Incident Month'] = df['Created Date'].dt.month
df['Incident Hour'] = df['Created Date'].dt.hour
df['Incident Weekday'] = df['Created Date'].dt.weekday_name
df.head()

,SR Type Code,SR Description,Owning Department,Method Received,Created Date,Zip Code,Latitude Coordinate,Longitude Coordinate,Incident Year,Incident Month,Incident Hour,Incident Weekday
0,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,2016-05-06 12:01:46,78703,30.296699,-97.768264,2016,5,12,Friday
1,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,2016-05-06 19:51:27,78745,30.212695,-97.801521,2016,5,19,Friday
2,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,2016-05-06 20:22:56,78745,30.212695,-97.801521,2016,5,20,Friday
3,SWSDEADA,ARR Dead Animal Collection,Austin Resource Recovery,Phone,2016-12-15 06:41:40,78741,30.230164,-97.731776,2016,12,6,Thursday
4,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,2016-05-06 17:03:45,78759,30.384989,-97.766471,2016,5,17,Friday


### Rename Columns

In [33]:
df = df.rename(columns={
    'SR Type Code':'Incident Type Code',
    'SR Description':'Incident Description',
    'Created Date':'Incident Date'
})
df.head()

,Incident Type Code,Incident Description,Owning Department,Method Received,Incident Date,Zip Code,Latitude Coordinate,Longitude Coordinate,Incident Year,Incident Month,Incident Hour,Incident Weekday
0,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,2016-05-06 12:01:46,78703,30.296699,-97.768264,2016,5,12,Friday
1,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,2016-05-06 19:51:27,78745,30.212695,-97.801521,2016,5,19,Friday
2,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,2016-05-06 20:22:56,78745,30.212695,-97.801521,2016,5,20,Friday
3,SWSDEADA,ARR Dead Animal Collection,Austin Resource Recovery,Phone,2016-12-15 06:41:40,78741,30.230164,-97.731776,2016,12,6,Thursday
4,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,2016-05-06 17:03:45,78759,30.384989,-97.766471,2016,5,17,Friday


In [34]:
# Verify we have all incidents from all years
df['Incident Year'].value_counts()

2018    134194
2015    132828
2016    124054
2014    123457
2017    100312
2019     99629
2013        20
Name: Incident Year, dtype: int64

### Save the clean data frame

In [35]:
!rm -rf 'clean_data/clean_austin_311.csv'
df.to_csv('clean_data/clean_austin_311.csv', index=False)